In [1]:
# set configurations
# from model.config import data_dir, weights_load_path

# base_dir = "."
# data_dir = "../../datasets/coco"  
# weights_load_path = "./model/weights/weight_stepLR_2.pth"
# activation_base_dir = "."


In [39]:
import json
from model.resnet18 import resnet18
from types import SimpleNamespace
import matplotlib.pyplot as plt
import numpy as np
from activation.visualize import visualize
from activation.utils import predict_label
import torch
from model.coco_dataset import get_test_coco_dataset_iter
from sacred_config import ex
import cv2

In [40]:

EXPERIMENT_LOCAL_CONFIG = "./local_config.json"
with open(EXPERIMENT_LOCAL_CONFIG) as f:
    json_dict = json.load(f)
param = SimpleNamespace(**json_dict)

In [41]:
model = resnet18(param.pretrain, param.class_ids, param.finetune,param.weights_load_path, param.end_epoch, 
                 param.start_epoch, param.checkpoints, param.save_weights_loc)

Using cache found in /Users/pawnesh/.cache/torch/hub/pytorch_vision_v0.6.0


In [42]:
model.layer1

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [43]:
# def visualize(images, titles, figsize, ncols, nrows, save_path, normalise=True):
#     """ Visualization of images """
#     fig, axes = plt.subplots(figsize=figsize, ncols=ncols, nrows=nrows, squeeze=False)
#     # if (len(images) != (ncols*nrows)) and (len(titles) != (ncols*nrows)):
#     #     return False
#     imgs = iter(images)
#     titles = iter(titles)
#     try:
#         for row_ind in range(nrows):
#             for col_ind in range(ncols):
#                 img = next(imgs)
#                 # Todo:
#                 if normalise:
#                     mean = np.array([0.485, 0.456, 0.406])
#                     std = np.array([0.229, 0.224, 0.225])
#                     img = std * img + mean
#                     img = np.clip(img, 0, 1)
#                 axes[row_ind, col_ind].imshow(img)
#                 axes[row_ind, col_ind].set_title(next(titles))
#     except Exception as err:
#         print(err)
#     finally:
#         fig.savefig(save_path)
#     return True

In [53]:

def prediction_visualization(model, class_ids, activation_save_path, num_of_cams=2):
    """ Visualize the activation maps of a prediction model. """

    len_of_labels = int(len(class_ids))
    nrows = 5
    start_ind, ind = 0, 0
#     data_to_visualize, labels_for_vis_data = get_visualization_data(model, get_coco_samples_per_class)

    for ind in range(0, len_of_labels, nrows):
        end_ind = start_ind + (nrows*(num_of_cams+1))
        print("start", start_ind)
        print("end", end_ind)
#         is_success = visualize(data_to_visualize[start_ind:end_ind], labels_for_vis_data[start_ind:end_ind],
#                                figsize=(10, 15), nrows=nrows, ncols=num_of_cams+1,
#                                save_path=f"{activation_save_path}/activation_map_{ind}.jpg")
        
        start_ind += nrows*(num_of_cams+1)

#     is_success = visualize(data_to_visualize[start_ind:], labels_for_vis_data[start_ind:], figsize=(10, 15),
#                            nrows=nrows, ncols=num_of_cams+1,
#                            save_path=f"{activation_save_path}/activation_map_{ind+1}.jpg")
    return True

In [54]:
prediction_visualization(model, param.class_ids, param.activation_save_path)

start 0
end 15
start 15
end 30
start 30
end 45
start 45
end 60
start 60
end 75


True

In [37]:




def get_coco_samples_per_class(number_of_classes, samples_per_class=1):
    """ Fetch samples for each class .
    # Todo: Extend the functionality to support param: samples_per_class.
    """
    images = []
    #images = torch.Tensor(number_of_classes, 3, 224, 224)
    #labels = torch.Tensor(number_of_classese)
    labels = []
    test_data_iter = get_test_coco_dataset_iter(param.class_ids, param.val_meta_file, 
                                                param.val_data_dir, param.batch_size, 
                                                param.num_workers)##
    visited_classes = []
    for data_batch, label_batch in test_data_iter:
        for data, label in zip(data_batch, label_batch):
            label = label.numpy().item()
            if label not in visited_classes:
                #torch.cat(data, out=images)
                images.append(data)
                visited_classes.append(label)
                labels.append(label)
                #torch.cat()
            if len(visited_classes) == number_of_classes:
                break
    return torch.stack(images), torch.Tensor(labels)


# def predict_label(model, data, num_of_cams=2):
#     """predict the label for an image."""
# #     if isinstance(data, np.ndarray):
# #         data = torch.from_numpy(data)
# #     data.unsqueeze_(0)
#     last_conv_layer = "layer4"
#     model.eval()
#     features_blobs = []

#     def last_conv_layer_hook(module, grad_input, grad_output):
#         features_blobs.append(grad_output.data.cpu().numpy())

#     model._modules.get(last_conv_layer).register_forward_hook(last_conv_layer_hook)
#     softmax_weight = np.squeeze(list(model.parameters())[-2].data.numpy())
#     #print(softmax_weight)
#     result = model(data)
#     result = torch.nn.functional.softmax(result, dim=1).data.squeeze()
#     result = torch.topk(result, k=2, dim=1)
#     return result, features_blobs, softmax_weight


def get_visualization_data(model, get_samples_func):
    """ Fetch Visualization data. input_image, cam1, cam2 ."""
    data_to_visualize = []
    labels_for_vis_data = []
    images, labels = get_samples_func(number_of_classes=24)##
    t_topk, features, softmax_weight = predict_label(model, images)
    probs, pred_label = t_topk
    probs, pred_label = probs.detach().numpy(), pred_label.detach().numpy()
    
    for each_img, each_label in zip(images.numpy(), labels.numpy()):
        
        each_img = each_img.numpy().transpose((1, 2, 0))
        # input data
        data_to_visualize.append(each_img)
        labels_for_vis_data.append(each_label)
        # cam1 data
        data_to_visualize.append(each_img)
        labels_for_vis_data.append(result.numpy().item())
        # cam2 data
        data_to_visualize.append(each_img)
        labels_for_vis_data.append(result.numpy().item())

    return data_to_visualize, labels_for_vis_data

In [38]:
imgs, labs = get_coco_samples_per_class(number_of_classes=24)

KeyboardInterrupt: 

In [10]:
imgs.shape

torch.Size([24, 3, 224, 224])

In [29]:
t_topk, features = predict_label(model, imgs)
probs, pred_label = t_topk

In [30]:
probs, pred_label = probs.detach().numpy(), pred_label.detach().numpy()

In [22]:
list(model.parameters())[-2][20].shape

torch.Size([512])

In [15]:
t1 = list(model.parameters())[-2][20]
t2 = list(model.parameters())[-3]

In [16]:
torch.topk(t1*t2, k=2).indices.numpy()

array([198, 404])

In [22]:


cam = features[0][1][198]
cam -= np.min(cam)
cam /= np.max(cam)
cam_img = np.uint8(255 * cam)


In [32]:
np.squeeze(pred_label)

array([13,  8, 12, 18,  0, 21, 19,  2, 20,  4,  4, 11, 22, 13,  0, 10,  6,
       21, 17,  6,  7, 16, 23, 14])

In [36]:
list(model.parameters())[-3].shape

torch.Size([512])

In [25]:
# main
prediction_visualization(model, param.class_ids, param.activation_save_path)

> <ipython-input-24-b40890421333>(40)predict_label()
     38     result = model(data)
     39     import pdb; pdb.set_trace()
---> 40     result = torch.topk(result, k=2, dim=-1)  ##
     41     return result
     42 

ipdb> result
tensor([[-3.4233e-01, -3.4654e+00, -2.0715e+00, -8.7725e-01, -2.3427e-01,
          5.3524e+00,  1.7179e+00,  1.3368e+00, -9.7993e-01, -9.8762e-01,
         -2.2510e+00, -1.1951e+00, -1.6797e+00, -4.2046e-01,  2.0662e+00,
         -1.2579e+00, -5.3066e-01, -1.1895e+00,  2.1218e-03,  5.3471e-01,
         -8.1082e-01,  1.4076e+00,  2.6985e+00,  1.2873e+00],
        [ 4.5497e+00, -8.1549e-01,  8.3232e-01,  2.0941e+00,  2.0961e-01,
          2.2897e+00, -6.9344e-01,  8.9430e-01,  1.1651e+00,  1.8754e-01,
         -7.0124e-01, -1.1833e+00, -1.6573e+00,  3.1535e-01, -5.8290e-01,
         -9.6892e-01, -2.6252e+00, -2.4093e+00, -1.7131e+00, -2.0215e-01,
          1.3241e-01, -4.6746e-01, -2.2333e-01,  4.1607e-01],
        [ 9.9945e-01, -1.2253e+00, -1.8894e+00, -1.1

ipdb> n
> <ipython-input-24-b40890421333>(41)predict_label()
     39     import pdb; pdb.set_trace()
     40     result = torch.topk(result, k=2, dim=-1)  ##
---> 41     return result
     42 
     43 

ipdb> result
torch.return_types.topk(
values=tensor([[ 5.3524,  2.6985],
        [ 4.5497,  2.2897],
        [ 3.9867,  3.7687],
        [ 2.4526,  2.4277],
        [ 7.4810,  3.9416],
        [ 2.3711,  2.2233],
        [ 4.6297,  3.3172],
        [ 2.2365,  2.1648],
        [11.5831,  3.5039],
        [ 7.6685,  2.8861],
        [ 6.2002,  4.4657],
        [ 3.4285,  2.4471],
        [ 9.3881,  6.1121],
        [ 8.1139,  4.3912],
        [ 7.0072,  3.1849],
        [ 8.6483,  6.4289],
        [ 5.3676,  2.3003],
        [ 3.3360,  3.2040],
        [10.6585,  2.2529],
        [ 6.4111,  3.3234],
        [ 4.6037,  3.4445],
        [ 2.9996,  2.9939],
        [ 6.8347,  4.9076],
        [ 8.1828,  4.3692]], grad_fn=<TopkBackward>),
indices=tensor([[ 5, 22],
        [ 0,  5],
        [ 

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [41]:
    model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  